Data preparing

In [1]:
def prepare(l: str):
    s = l.strip().lower()

    for numChar in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        s = s.replace(numChar, '', -1)

    for c in ['"', "'", '(', ')', '*', ',', '-', ':', ]:
        s = s.replace(c, '', -1)

    for endChar in ['?', '!', ';']:
        s = s.replace(endChar, '.', -1)

    s = " ".join(s.split())
    return s.split('.')


file = open('data/The Brothers Karamazov cropped.txt', encoding='UTF-8')
text = ''
sentences = []

for line in file.readlines():
    line_sentences = prepare(line)
    for sentence in line_sentences:
        striped = sentence.strip()
        if len(striped) != 0:
            text += striped + '.'
            sentences.append(striped + '.')


In [2]:
chars = sorted(set(text))
alphabet_size = len(chars)
print("alphabet = ", chars, sep=' ')
print("alphabet size after preparing = ", alphabet_size)
assert (text.count('.') == len(sentences))
print("sentences count = ", len(sentences))
print("doubled spaces count = ", text.count('  '))

alphabet =  [' ', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
alphabet size after preparing =  28
sentences count =  4684
doubled spaces count =  0


In [3]:
sentences[:150]

['part i.',
 'book i.',
 'the history of a family.',
 'chapter.',
 'fyodor pavlovitch karamazov.',
 'alexey fyodorovitch karamazov was the third son of fyodor pavlovitch karamazov a landowner well known in our district in his own day and still remembered among us owing to his gloomy and tragic death which happened thirteen years ago and which i shall describe in its proper place.',
 'for the present i will only say that this landowner for so we used to call him although he hardly spent a day of his life on his own estate was a strange type yet one pretty frequently to be met with a type abject and vicious and at the same time senseless.',
 'but he was one of those senseless persons who are very well capable of looking after their worldly affairs and apparently after nothing else.',
 'fyodor pavlovitch for instance began with next to nothing.',
 'his estate was of the smallest.',
 'he ran to dine at other mens tables and fastened on them as a toady yet at his death it appeared that he h

In [4]:
print(text[:40_000])

part i.book i.the history of a family.chapter.fyodor pavlovitch karamazov.alexey fyodorovitch karamazov was the third son of fyodor pavlovitch karamazov a landowner well known in our district in his own day and still remembered among us owing to his gloomy and tragic death which happened thirteen years ago and which i shall describe in its proper place.for the present i will only say that this landowner for so we used to call him although he hardly spent a day of his life on his own estate was a strange type yet one pretty frequently to be met with a type abject and vicious and at the same time senseless.but he was one of those senseless persons who are very well capable of looking after their worldly affairs and apparently after nothing else.fyodor pavlovitch for instance began with next to nothing.his estate was of the smallest.he ran to dine at other mens tables and fastened on them as a toady yet at his death it appeared that he had a hundred thousand roubles in hard cash.at the sa

LSTM

In [5]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

char_to_int

{' ': 0,
 '.': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [6]:
int_to_char

{0: ' ',
 1: '.',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'j',
 12: 'k',
 13: 'l',
 14: 'm',
 15: 'n',
 16: 'o',
 17: 'p',
 18: 'q',
 19: 'r',
 20: 's',
 21: 't',
 22: 'u',
 23: 'v',
 24: 'w',
 25: 'x',
 26: 'y',
 27: 'z'}

In [7]:
def char_one_hot(char):
    return [int(i == char_to_int[char]) for i in range(alphabet_size)]


def line_one_hot(sent):
    return [char_one_hot(char) for char in sent]


text_one_hot = [line_one_hot(sent) for sent in sentences]

print(sentences[0][0], " -> ", text_one_hot[0][0])
print(sentences[0][1], " -> ", text_one_hot[0][1])
print(sentences[0][2], " -> ", text_one_hot[0][2])
print(sentences[0][3], " -> ", text_one_hot[0][3])

p  ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
a  ->  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
r  ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
t  ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [8]:
import numpy as np

window_size = 50
dataX = []
dataY = []

for sent in text_one_hot:
    if len(sent) > window_size:
        for i in range(0, len(sent) - window_size):
            dataX.append(sent[i:i + window_size])
            dataY.append(sent[i + window_size])

X, Y = np.array(dataX), np.array(dataY)
print(X.shape, Y.shape)

(146219, 50, 28) (146219, 28)


In [9]:
import tensorflow as tf

model_lstm = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(window_size, alphabet_size)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(alphabet_size, activation='softmax')
])

model_lstm.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
model_lstm.fit(X, Y, epochs=10)

Epoch 1/10
4570/4570 [==============================] - 93s 20ms/step - loss: 2.2296
Epoch 2/10
4570/4570 [==============================] - 91s 20ms/step - loss: 1.8894
Epoch 3/10
4570/4570 [==============================] - 91s 20ms/step - loss: 1.7567
Epoch 4/10
4570/4570 [==============================] - 90s 20ms/step - loss: 1.6678
Epoch 5/10
4570/4570 [==============================] - 91s 20ms/step - loss: 1.6057
Epoch 6/10
4570/4570 [==============================] - 91s 20ms/step - loss: 1.5568
Epoch 7/10
4570/4570 [==============================] - 93s 20ms/step - loss: 1.5180
Epoch 8/10
4570/4570 [==============================] - 94s 21ms/step - loss: 1.4844
Epoch 9/10
4570/4570 [==============================] - 95s 21ms/step - loss: 1.4576
Epoch 10/10
4570/4570 [==============================] - 94s 21ms/step - loss: 1.4313


In [199]:
import random


def random_lstm_predict():
    start_index = random.randint(0, len(text) - window_size - 1)
    string = text[start_index:start_index + window_size]
    string = line_one_hot(string)
    return lstm_predict(string)


def lstm_predict(string):
    while True:
        predictions = model_lstm.predict(np.array([string]))[0]
        predictions = np.exp(np.log(predictions))
        predictions = predictions / np.sum(predictions)
        p = np.random.multinomial(1, predictions, 1)
        index = np.argmax(p)
        char = int_to_char[index]
        if char == '.':
            result = ""
            for char in string:
                for i in range(alphabet_size):
                    if char[i] == 1:
                        result += int_to_char[i]
            return result
        else:
            string.append(char_one_hot(char))


In [200]:
random_lstm_predict()

1/1 [==============================] - 0s 10ms/step


'ill you get out of me.it must be noted again that iven his dere you tinch pithors emendeal obsilation of the eldersthe'

Markov chain

In [174]:
last_n = 8

graph = dict()
for i in range(len(text) - last_n - 1):
    l = text[i: i + last_n]
    r = text[i + 1: i + last_n + 1]
    if l not in graph:
        graph[l] = dict()
    if r not in graph[l]:
        graph[l][r] = 1
    else:
        graph[l][r] += 1


In [175]:
graph[list(graph.keys())[56]]

{'ovitch k': 3,
 'ovitch f': 3,
 'ovitch i': 7,
 'ovitch w': 29,
 'ovitch d': 8,
 'ovitch y': 2,
 'ovitch a': 17,
 'ovitch l': 4,
 'ovitch j': 2,
 'ovitch u': 4,
 'ovitch r': 2,
 'ovitch b': 7,
 'ovitch m': 5,
 'ovitch t': 6,
 'ovitch c': 8,
 'ovitch s': 12,
 'ovitch h': 9,
 'ovitch o': 3,
 'ovitch g': 2,
 'ovitch p': 5,
 'ovitch n': 1,
 'ovitch e': 1}

In [176]:
probabilities = dict()
for key, value in graph.items():
    count = 0
    probabilities[key] = dict()
    for k, v in value.items():
        count += v
    for k, v in value.items():
        probabilities[key][k] = v / count


In [177]:
probabilities[list(probabilities.keys())[56]]

{'ovitch k': 0.02142857142857143,
 'ovitch f': 0.02142857142857143,
 'ovitch i': 0.05,
 'ovitch w': 0.20714285714285716,
 'ovitch d': 0.05714285714285714,
 'ovitch y': 0.014285714285714285,
 'ovitch a': 0.12142857142857143,
 'ovitch l': 0.02857142857142857,
 'ovitch j': 0.014285714285714285,
 'ovitch u': 0.02857142857142857,
 'ovitch r': 0.014285714285714285,
 'ovitch b': 0.05,
 'ovitch m': 0.03571428571428571,
 'ovitch t': 0.04285714285714286,
 'ovitch c': 0.05714285714285714,
 'ovitch s': 0.08571428571428572,
 'ovitch h': 0.06428571428571428,
 'ovitch o': 0.02142857142857143,
 'ovitch g': 0.014285714285714285,
 'ovitch p': 0.03571428571428571,
 'ovitch n': 0.007142857142857143,
 'ovitch e': 0.007142857142857143}

In [181]:
def random_markov_chain_predict():
    start_index = random.randint(0, len(text) - last_n - 1)
    string = text[start_index:start_index + last_n]
    return markov_chain_predict(string)


def markov_chain_predict(string):
    result = string
    while True:
        string = string[-last_n:]
        if string not in probabilities:
            return result + '.'
        prediction = max(probabilities[string], key=probabilities[string].get)
        new_char = prediction[-1]
        result += new_char
        if new_char == '.':
            return result
        string += prediction


In [182]:
markov_chain_predict("fyodor pavlovitch")

'fyodor pavlovitch was a divinity student leaving mitya a child of three years old in her hands before the inmates being a regular roman nose he used to say that i should be no sin in it if i had only just because i spoke of eating gudgeon a day and happy faces.'

In [188]:
random_markov_chain_predict()

' where the inmates being a regular roman nose he used to say that i should be no sin in it if i had only just because i spoke of eating gudgeon a day and happy faces.'